## Scaleway Object Storage - Testing

https://www.simplecto.com/using-django-and-boto3-with-scaleway-object-storage/

* `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY` can be obtained from the [credentials control panel](https://console.scaleway.com/project/credentials) under API Keys.
* `AWS_STORAGE_BUCKET_NAME` is the name of the bucket you create on [objects administration page](https://console.scaleway.com/object-storage/buckets)
* `AWS_DEFAULT_ACL` is set to public-read so that the objects can be pulled from a URL without any access keys or time-limited signatures.
* `AWS_S3_REGION_NAME` and `AWS_S3_ENDPOINT_URL` should be configured so that `boto3` knows to point to the Scaleway resources.

All of these are references in the Scaleways docs on Object Storage.

### Resources:
* https://www.scaleway.com/en/docs/object-storage-feature/
* https://www.scaleway.com/en/docs/how-to-migrate-object-storage-buckets-with-rclone/
* https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html

In [4]:
import tomli
from pathlib import Path

In [5]:
APP_CONFIG_FILE = 'app_config.toml'

In [7]:
with open(Path('..')/APP_CONFIG_FILE, encoding="utf-8") as f:
    app_config = tomli.load(f)

In [16]:
AWS_ACCESS_KEY_ID = app_config['AWS']['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = app_config['AWS']['AWS_SECRET_ACCESS_KEY']
AWS_STORAGE_BUCKET_NAME = app_config['AWS']['AWS_STORAGE_BUCKET_NAME']
AWS_DEFAULT_ACL = app_config['AWS']['AWS_DEFAULT_ACL']
AWS_S3_REGION_NAME = app_config['AWS']['AWS_S3_REGION_NAME']
AWS_S3_ENDPOINT_URL =  app_config['AWS']['AWS_S3_ENDPOINT_URL']

In [20]:
import boto3
from pathlib import Path

In [38]:
#from os import path, makedirs
#from botocore.exceptions import ClientError
#from boto3.exceptions import S3TransferFailedError

In [21]:
s3 = boto3.client('s3', 
        region_name=AWS_S3_REGION_NAME, 
        endpoint_url=AWS_S3_ENDPOINT_URL, 
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

In [35]:
def download_s3_folder(s3, bucket_name, s3_folder, local_dir=None):
    filecount = 0
    files = []
    if not local_dir.exists():
        Path.makedirs(local_dir)
    bucket_list=s3.list_objects(Bucket=bucket_name)['Contents']
    for s3_key in bucket_list:
        s3_object = s3_key['Key']
        if not s3_object.endswith("/"):
            filepath = Path(local_dir)/s3_object
            s3.download_file(bucket_name, s3_object, filepath.as_posix())
            filecount+=1
            files.append(s3_object)
        else:
            if not (Path(local_dir)/s3_object).exists():
                Path.makedirs(Path(local_dir)/s3_object)
    return filecount, files

In [36]:
filecount, files = download_s3_folder(s3, AWS_STORAGE_BUCKET_NAME, 'emmaus-walking', Path.home()/'tmp')

In [37]:
print(str(filecount) + ' files downloaded')
print(files)

9 files downloaded
['emmaus-walking/GWW/2020-05-11-095108-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit', 'emmaus-walking/GWW/2020-05-18-110321-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit', 'emmaus-walking/GWW/2020-05-25-095315-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit', 'emmaus-walking/GWW/2020-05-25-115046-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit', 'emmaus-walking/GWW/2020-06-01-094350-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit', 'emmaus-walking/GWW/2020-06-15-084936-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit', 'emmaus-walking/GWW/2020-06-22-085751-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit', 'emmaus-walking/GWW/2021-03-12-085528-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit', 'emmaus-walking/GWW/2021-04-03-084312-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit']
